# Calculation of volume constraints for Si-composite anodes
Jan Petter Maehlen, IFE, 2016

[EDIT 31.11.2018]: updated to Python3, removed names of electrodes

Acknowledgements: Elkem and Research Council of Norway

In [ ]:
# from IPython.display import display, Math, Latex
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## *Key parametres*
* Porousity change
* Volume expansion
* Si fraction
* Lithiation of Si
* Lithiation of C [graphite]
* Available volume

# 1. Background and introduction
* For a full cell using silicon-graphite composite as anode, areal loading of the anode needs to be merged with the areal loading of the cathode. Usually, the anode capacity should be higher than the cathode to prevent too low voltages vs. Li/Li+ during charging (lithium plating).

* Silicon expands during lithiation (280% for full lithiation , ref. xx). For a workable cell, we need a electrode configuration that allows the expansion without compromising the porosity (needed for flow of electrolyte). 

# 2. Definitons and relations
Our model anode is divided into 4 main parts:
1. Silicon
2. Graphite
3. Non-active (dead) material
4. Void

One can in addition allow for some "breathing" of the anode during cycling ("Room for expansion"). Wether or not this is realistic depends on the battery type (assumption), as it could be that some space is needed for manufacturing purpuses (*e.g.* easy insertion of electrolyte after assembly).

<img src="SiliconExpansion_fig_001.png">

**Fig. 1.** *Schematic view of Si-Graphite composite anode*

### Lithiation fraction
Defining a normalized lithiation parameter, *x*

*x<sub>i</sub>: degree of lithiation for spieces i*

Condition:

$$x \in [0,1] $$

*i.e.*

*x = 1* for a fully lithiated anode,

*x = 0* for a fully delithiated anode.

<img src="SiliconExpanded_fig_001.png">

**Fig. 2.** *Schematic view of Si-Graphite composite anode, in delithiated (x = 0) and lithiated (x = n, 0 < n < 1) state*

### :- Assumption set 1
a.
*x<sub>S</sub> = x<sub>G</sub> = x<sub>A</sub> = x* (degree of lithiation same for all spieces)

or

b.
*x<sub>i</sub> = F<sub>i</sub>(f)* (lithiation of spieces *i* given by a pre-defined function)



### Capacities
*C<sub>i</sub>: capacity for spieces i in [mAh/g]*


**maxC**<sub>i</sub>*: max capacity for spieces i in [mAh/g]*


*C<sub>S,x</sub> = x **maxC**<sub>S</sub>*

*C<sub>G,x</sub> = x **maxC**<sub>G</sub>*

*C<sub>A,x</sub> = x (**maxC**<sub>S</sub> + **maxC**<sub>G</sub>)*

### Volumes

*V<sub>A</sub> = V<sub>S</sub> + V<sub>G</sub> + V<sub>D</sub> + V<sub>V</sub>*

*V<sub>A,0</sub> = V<sub>A</sub>*

*V<sub>A,n</sub> = V<sub>A</sub> + V<sub>E</sub> = eps V<sub>A</sub>*

*eps = fractional room for expansion*                            [min(eps) = 1]

*eps = 1 + V<sub>E</sub> /  V<sub>A</sub>*

*V<sub>E</sub> = (eps - 1) V<sub>A</sub>*

*V<sub>A</sub> = (1 - eps) V<sub>E</sub>*


### Densities and fractions
*f<sub>i</sub>: fraction of spieces i*

*R<sub>i</sub>: density of spieces i*

*M<sub>A</sub>: mass of electrode*


*V<sub>i</sub> = (f<sub>i</sub> M<sub>A</sub>) / R<sub>i</sub>*

### :- Assumption set 2
*M<sub>A</sub> = 1* (normalized mass)

*V<sub>i</sub> = f<sub>i</sub> / R<sub>i</sub>*

### Expansion factors
*e<sub>i</sub>: expansion coeff for spieces i*

*V<sub>S,x</sub> = (1 + e<sub>S</sub> x) V<sub>S</sub>*

*V<sub>G,x</sub> = (1 + e<sub>G</sub> x) V<sub>G</sub>*

*V<sub>D,x</sub> = V<sub>D</sub>*

*V<sub>A,x</sub> = eps V<sub>A</sub>*

*V<sub>V,x</sub> = V<sub>A,x</sub> - [V<sub>S,x</sub> + V<sub>G,x</sub> + V<sub>D,x</sub>]*

*V<sub>V,x</sub> = eps V<sub>A</sub> - [ (1 + e<sub>S</sub> x) V<sub>S</sub> + (1 + e<sub>G</sub> x) V<sub>G</sub> + V<sub>D</sub>]*

### Porosity
*P = V<sub>V</sub> / V<sub>A</sub>*

*P<sub>x</sub> = V<sub>V,x</sub> / V<sub>A,x</sub>*



Gives:


*P<sub>x</sub> = (eps V<sub>A</sub> - [ (1 + e<sub>S</sub> x) V<sub>S</sub> + (1 + e<sub>G</sub> x) V<sub>G</sub> + V<sub>D</sub>]) / eps V<sub>A</sub>*

# 3. Calculations

## 3.1. Parametres
### Coefficients

In [ ]:
# Needed input prms

# expansion coeffs
expansion_coeff_si       = 2.8 
expansion_coeff_graphite = 0.1
expansion_coeff_dead     = 0.0

# densities
rho_si = 2.329           # g/ccm
rho_graphite = 2.1       # g/ccm
rho_dead = 1.8           # g/ccm

# max capacities
maxcap_si = 3579.0       # mAh/g
maxcap_graphite = 372.0  # mAh/g
maxcap_dead = 0.0        # mAh/g




### Input parametres (defaults)

In [ ]:
mass_fraction_silicon  = 0.6
mass_fraction_graphite = 0.3
mass_fraction_dead     = 0.1

radius = 0.5*1.5

eps = 1.0

# cut-off capacities
max_cutoff_cap_si = 3579.0       # mAh/g
max_cutoff_cap_graphite = 372.0  # mAh/g
max_cutoff_cap_dead = 0.0        # mAh/g

porosity = 0.5 # initial porosity
porosity_limit = 0.0 # final limit for porosity

## 3.2. Functions

In [ ]:
def find_area(radius):
    return np.pi*radius*radius


In [ ]:
def density_all(densities, mass_fractions, porosity = 0.0):
    
    Total_mass = np.sum(mass_fractions)
    
    Total_volume = 0.0
    for d,m in zip(densities, mass_fractions):
        #print "density: %f\nfraction: %f" % (d,m)
        Total_volume += m/d
    try:
        Total_volume = Total_volume/(1-porosity)
    except ZeroDivisionError:
        print("ERROR! You have a porosity of 1.0 (meaning your electrode contains nothing)")
        return None
    
    Density = Total_mass/Total_volume
    #print "Density: %f" % (Density)
    return Density

In [ ]:
def porosity_from_densities(d_measured,d_theory = 1.0):
    return (1-(d_measured/d_theory))


In [ ]:
a = find_area(radius)
print(f"Area: {a} cm^2");

In [ ]:
def createX(max_cutoff_caps, max_caps, limits = [0.0, 1.0], bits = 100, option = None):
    """This function will (when finnished) make a four x-arrays with adjusted x-values depending
    on type of material and maximum cutoff capacities. By ysing this to make the lithiation fractions,
    iterations can be performed in the full x-range (the function takes care of limits)
    
    i.e.
    X = createX(max_cutoff_caps)
    for x in X:
        V = volume_vs_x(x, volumes, expansion_coeffs, eps)
    
    returns numpy array X = [[x1_0,x2_0,x3_0],[x1_1,x2_1,x3_1] ...] where X contains 'bits' elements (linspace)"""
    
    _x1_max = x_cutoff(max_cutoff_caps[0],max_caps[0])
    _x2_max = x_cutoff(max_cutoff_caps[1],max_caps[1])
    _x3_max = x_cutoff(max_cutoff_caps[2],max_caps[2])
    

    _x_max = [_x1_max,_x2_max,_x3_max]
    _x = np.linspace(limits[0], limits[1], bits)
    
    X = []
    
    for x in _x:
        b = []
        for x_max in _x_max:
            if x<x_max:
                b.append(x)
            else:
                b.append(x_max)
        X.append(b)

    X = np.array(X)
    if option == "scalingSi":
        X[:,0] = np.linspace(limits[0], _x1_max, bits)
    
            
    return np.array(X)


In [ ]:
def capacity(x,c_max):
    return x*c_max

In [ ]:
def capacity_all(x,max_caps, volumes, densities):
    """returns the total capacity for the electrode in mAh/g(tot)"""
    try:
        _x = x
        x_s,x_g,x_d = _x
    except TypeError:
        print("capacity_all: x is not list - setting same value for all materials")
        _x = [x,x,x]
        
    cap_all = 0.0
    for x_i,mc_i,v_i,d_i in zip(_x,max_caps,volumes,densities):
        cap_all += x_i*mc_i*v_i*d_i
    return cap_all

    

In [ ]:
def x_cutoff_total_even(c,max_caps):
    print("not implemented yet")
    return None

In [ ]:
def x_cutoff(c,c_max):
    try:
        cutoff = c/c_max
    except ZeroDivisionError:
        cutoff = 0.0
    return cutoff

In [ ]:
def volume_from_fR(f,R):
    return f/R
    

In [ ]:
def volume_vs_x(x, volumes, expansion_coeffs, eps):
    # List order = [Si,Graphite,Dead,Void]
    try:
        x_s,x_g,x_d = x
    except TypeError:
        print("volume_vs_x : x is not list - setting same value for all materials")
        x_s = x
        x_g = x
        x_d = x
        
    e_s,e_g,e_d = expansion_coeffs
    V_s,V_g,V_d,V_v = volumes
    
    V_a_x = eps*np.sum(volumes)

    V_s_x = (1+e_s*x_s)*V_s
    V_g_x = (1+e_g*x_g)*V_g
    V_d_x = (1+e_d*x_d)*V_d
    V_v_x = V_a_x - np.sum([V_s_x,V_g_x,V_d_x])
    
    return [V_s_x, V_g_x, V_d_x, V_v_x]

In [ ]:
def pore_volume_from_porosity(porosity, volumes):
    V_materials = np.sum(volumes[0:3])
    V_void      = porosity*V_materials / (1-porosity)
    return V_void

In [ ]:
def porosity_from_volumes(volumes):
    # P = V_void / V_anode
    V_void  = volumes[3]
    V_anode = np.sum(volumes)
    return V_void / V_anode

In [ ]:
def calc_loading(capacity,density,thickness):
    return capacity*density*thickness

In [ ]:
def calc_needed_thickness(target_capacity, target_loading, density):
    return target_loading/(target_capacity*density)

In [ ]:
# defining a convinence-constants for converting to-from cm - micron, g - mg
to_micron = 10000.0
to_mg     = 1000.0

## 3.3. Setup
Putting parametres in lists to reduce typing (and possible implement numpy arrays later for speed)

In [ ]:
# Silicon, Graphite, Dead, [void]

mass_fractions   = [mass_fraction_silicon, mass_fraction_graphite,mass_fraction_dead]
expansion_coeffs = [expansion_coeff_si, expansion_coeff_graphite, expansion_coeff_dead]
densities        = [rho_si, rho_graphite, rho_dead]
max_caps         = [maxcap_si, maxcap_graphite, maxcap_dead]
max_cutoff_caps  = [max_cutoff_cap_si, max_cutoff_cap_graphite, max_cutoff_cap_dead]


In [ ]:
assert round(np.sum(mass_fractions),2) == 1.00

## 3.4. Simple calculations and tests

### Simple calculations for thickness estimations

In [ ]:
# calculating needed thickness to match given demand on loading
d = density_all(densities, mass_fractions, porosity=0.5)
c = 522.0 # mAh/g(tot)
t = 20.0 # micron
t2 = t/to_micron

print(90*"-")
print("Calculating loading based on c: %4.1f mAh/g t: %4.1f micron d: %5.3f g/ccm" % (c,t,d))
print(f"Loading: {calc_loading(c,d,t2)} mAh/cm2")

l = 2.5
print (90*"-")
print("Calculating needed thickness based on c: %4.1f mAh/g l: %3.2f mAh/cm2 d: %5.3f g/ccm" % (c,l,d))
print(f"Thickness: {calc_needed_thickness(c,l,d)*to_micron} micron")
print(90*"-")

### Simple calculations of porosity

In [ ]:
# calculating the theoretical density
d_theory   = density_all(densities, mass_fractions, porosity=0.0)


# finding the measured density from mass, radius, and thickness
mass      = 2.9      # mg
radius    = 0.5*1.5  # cm (diameter of electrode is 1.5 cm)
thickness = 20.0     # micron

m2   = mass/to_mg
area = find_area(radius)
t2   = thickness/to_micron

d_measured= m2/(area*t2)

# calculating the porosity
p_calculated = porosity_from_densities(d_measured,d_theory) # = (1-d_measured/d_theory)

# reporting
txt = "\nCalculating porosity of electrodes based on measured density"
print(txt.upper())
print(90*"-")
print("Theoretical density: %5.3f g/ccm" % (d_theory,))
print("Measured density:    %5.3f g/ccm" % (d_measured,))
print("Calculated porosity: %5.3f" % (p_calculated,))
print(90*"-")

# check
assert round(d_measured,3) == round(density_all(densities, mass_fractions, porosity=p_calculated),3)

### Simple calculation of density

In [ ]:
# finding density for "dead" material

def molecular_mass(chemical_formula_dict):
    # Molecular mass dict
    mm = dict()
    mm["K"] = 39.098
    mm["O"] = 15.999
    mm["H"] = 1.008
    mm["C"] = 12.011

    molMass = 0.0
    for chemical, number in chemical_formula_dict.items():
        _molMass = mm[chemical.upper()]
        molMass += (number * _molMass)
        txt = "Chem: %2s - Number: %3.2f - Molecular mass: %6.3f g/mol" % (chemical, float(number), _molMass)
        print(txt)
    txt = "  ->  %6.3f g/mol" % (molMass)
    print(txt)
    return molMass

# ---defining chemicals---------------------------------------------
citric_acid_cf_dict = dict()
citric_acid_cf_dict["C"] = 6
citric_acid_cf_dict["H"] = 8
citric_acid_cf_dict["O"] = 7

potassium_hydroxide_cf_dict = dict()
potassium_hydroxide_cf_dict["K"] = 1
potassium_hydroxide_cf_dict["H"] = 1
potassium_hydroxide_cf_dict["O"] = 1

mMass_citric_acid = molecular_mass(citric_acid_cf_dict)
mMass_potassium_hydroxide = molecular_mass(potassium_hydroxide_cf_dict)

# ---prms----------------------------------------------------------
d_cmc            = 1.6
d_cb             = 2.1

d_KOH            = 2.12
d_citric_acid    = 1.66

_molLitre_KOH     = 0.173 # given by mixing the buffer
_molLitre_acid    = 0.074 # given by mixing the buffer

# ---input---------------------------------------------------------
f_cmc              =  5.0
f_cb               =  5.0
f_buffer_chemicals = 13.0

# ---calc----------------------------------------------------------

_f_KOH            = _molLitre_KOH * mMass_potassium_hydroxide
_f_citric_acid    = _molLitre_acid * mMass_citric_acid
_f_buffer         = _f_KOH + _f_citric_acid

f_KOH = _f_KOH / _f_buffer
f_citric_acid = _f_citric_acid / _f_buffer

d_buffer_chemicals = density_all([d_KOH, d_citric_acid], [f_KOH, f_citric_acid], porosity = 0.0)

_f_all_sum  = f_cmc + f_cb + f_buffer_chemicals
_f_all   = [f_cmc/_f_all_sum, f_cb/_f_all_sum, f_buffer_chemicals/_f_all_sum]
_d_all  = [d_cmc, d_cb, d_buffer_chemicals]

d_dead  = density_all(_d_all, _f_all, porosity = 0.0)
print( 90*"-")
print( "Density of 'dead' material: %6.3f g/ccm" % (d_dead))
print( 90*"-")






### Simple calculation of porosity and needed loading for "is030"

In [ ]:
# electrode: 20151113_is030_10_cc
# finding porosity
mass_is030_10      = 1.271 # mg
thickness_is030_10 = 10.0 # micron
radius             = 0.5*1.5  # cm (diameter of electrode is 1.5 cm)

mass_is030_10      = mass_is030_10/to_mg
area_is030_10      = find_area(radius)
thickness_is030_10 = thickness_is030_10/to_micron

f_silicon  = 0.522
f_graphite = 0.13
f_dead     = 1.0 - (f_silicon + f_graphite)
mass_fractions_is030_10 = [f_silicon,f_graphite,f_dead]

d_measured_is030_10 = mass_is030_10/(area_is030_10*thickness_is030_10)
d_theory_is030_10   = density_all(densities, mass_fractions_is030_10, porosity=0.0)

# calculating the porosity
p_calculated = porosity_from_densities(d_measured_is030_10,d_theory_is030_10)

# calculating needed thickness to match given demand on loading
d = density_all(densities, mass_fractions_is030_10, porosity=p_calculated)
c = f_silicon*1000.0 # mAh/g(tot)
l = 2.5   # mAh/cm2

t = calc_needed_thickness(c,l,d)*to_micron

# reporting
print( "\nCalculating porosity of electrodes based on measured density".upper())
print( 90*"-")
print( "Name:                is030_10")
print( 40*"-")
print( "Mass of electrode:   %5.3f mg" % (mass_is030_10,))
print( "Thickness:           %5.3f mg" % (thickness_is030_10,))
print( "Area:                %5.3f mg" % (area_is030_10,))
print( "Fraction Si:         %5.3f mg" % (f_silicon,))
print( "Fraction Graphite:   %5.3f mg" % (f_graphite,))
print( "Fraction Dead:       %5.3f mg" % (f_dead,))
print( 40*"-")
print( "Theoretical density: %5.3f g/ccm" % (d_theory_is030_10,))
print( "Measured density:    %5.3f g/ccm" % (d_measured_is030_10,))
print( "Calculated porosity: %5.3f" % (p_calculated,))
print( 90*"-")
print()
print( "\nCalculating needed thickness for given porosity etc.".upper())
print( 90*"-")
print( "Total capacity:      %5.3f mg" % (c,))
print( "Target loading:      %5.3f mAh/cm2" % (l,))
print( "Calc. thickness:     %5.3f mg" % (t,))
print( 90*"-")


### Visalisation
An intersting visualisation is how the thickness requirment changes as a function of the capacity cut-off.

In [ ]:
c0 = 320.0  # mAh/g(Si)
c1 = 3600.0 # mAh/g(Si)
l  = 2.5    # mAh/cm2

capacities = np.linspace(c0, c1, 100)
porosities = np.linspace(0.2,0.8,7) # could equally well have used a "hard-coded" list here

fname =r"C:\Scripting\MyFiles\notebooks\ElectrodeCalculations\SiliconExpansion\LvsC.png"
fname_Scaled =r"C:\Scripting\MyFiles\notebooks\ElectrodeCalculations\SiliconExpansion\LvsC_scaled.png"

_d = [density_all(densities, mass_fractions_is030_10, porosity=p) for p in porosities]

for d,p in zip(_d, porosities): 
    t = []
    for c in capacities:
        t.append(calc_needed_thickness(f_silicon*c,l,d)*to_micron)
    
    plt.plot(capacities,t, label = "p = %s" % p)
plt.xlabel("Capacity cut-off (mAh/g(Si))")
plt.ylabel("Thickness (micron)")
txt = "Loading:   %3.2f mAh/cm2\n" % (l)
txt+= "Mass%% Si: %3.2f\n" % (f_silicon)
txt+= "Mass%% G : %3.2f" % (f_graphite)

plt.grid()
plt.legend()

# creating and saving a "zoomed" figure
plt.xlim(1000, 2000)
plt.ylim(0, 150)
plt.savefig(fname_Scaled, dpi=500)

# creating, saving, and showing figure
plt.axhline(y = 100, linestyle = "--")
plt.axvline(x = 372, linestyle = "--")
plt.text(1000,200,txt,style='italic')
plt.xlim(300, 3620)
plt.ylim(0, 350)
plt.savefig(fname, dpi=500)

print( "files saved to",)
print( fname)
print( fname_Scaled)
    


### Fraction tests

In [ ]:
# Testing X-generation
cutoff_Si = 1000.0
max_cutoff_caps_test = [cutoff_Si, max_cutoff_cap_graphite, max_cutoff_cap_dead]
print( "cut-offs:", max_cutoff_caps_test)
x =  createX(max_cutoff_caps_test, max_caps, limits = [0.0, 1.0], bits = 100, option=None)


# plotting results
x_Si = x[:,0]
x_Graphite = x[:,1]
x_Void = x[:,2]

# individual plots
plt.figure()
plt.subplot(131)
plt.plot(x_Si, label = "Si")
plt.legend()
plt.subplot(132)
plt.plot(x_Graphite, label = "G")
plt.legend()
plt.subplot(133)
plt.plot(x_Void, label = "V")
plt.legend()

# same axes
plt.figure()
ax = plt.subplot(111)
ax.plot(x_Si, label = "Si")
ax.plot(x_Graphite, label = "Graphite")
ax.plot(x_Void, label = "Void")
plt.xlabel("bit")
plt.ylabel("lithiation fraction")
plt.ylim([-0.1,1.1])
plt.legend()
print()

In [ ]:
# Testing X-generation
# option scalingSi

cutoff_Si = 1000.0
max_cutoff_caps_test = [cutoff_Si, max_cutoff_cap_graphite, max_cutoff_cap_dead]
print( "cut-offs:", max_cutoff_caps_test)
x =  createX(max_cutoff_caps_test, max_caps, limits = [0.0, 1.0], bits = 100, option="scalingSi")


# plotting results
x_Si = x[:,0]
x_Graphite = x[:,1]
x_Void = x[:,2]

plt.figure()
plt.subplot(131)
plt.plot(x_Si, label = "Si")
plt.legend()
plt.subplot(132)
plt.plot(x_Graphite, label = "G")
plt.legend()
plt.subplot(133)
plt.plot(x_Void, label = "V")
plt.legend()


# same axes
plt.figure()
ax = plt.subplot(111)
ax.plot(x_Si, label = "Si")
ax.plot(x_Graphite, label = "Graphite")
ax.plot(x_Void, label = "Void")
plt.xlabel("bit")
plt.ylabel("lithiation fraction")
plt.ylim([-0.1,1.1])
plt.legend()
print

In [ ]:
# checking what the cut-off in x will be for given cut-off in Si capacity
c_silicon_cutoff = 1000.0
x_silicon_cutoff = x_cutoff(c_silicon_cutoff,maxcap_si)
print( "cut-off for %f mAh/g(Si) is %f" % (c_silicon_cutoff, x_silicon_cutoff))

## 4. Simulations of anode expansion

### Initialisation

In [ ]:
porosity = 0.5 # initial porosity
porosity_limit = 0.0 # final limit for porosity

In [ ]:
# need to make the initial volume list
volume_silicon_0 = volume_from_fR(mass_fraction_silicon,rho_si)
volume_graphite_0 = volume_from_fR(mass_fraction_graphite,rho_graphite)
volume_dead_0 = volume_from_fR(mass_fraction_dead,rho_dead)
volume_void_0 = np.NAN # not really needed

In [ ]:
# calculating the initial pore volume (void volume)
volume_void_0 = pore_volume_from_porosity(porosity, [volume_silicon_0,volume_graphite_0, volume_dead_0])
volumes = [volume_silicon_0,volume_graphite_0,volume_dead_0, volume_void_0]

In [ ]:
print( "volumes:", volumes)
print( "sum of volumes (anode volume):", np.sum(volumes))
print( "Input porosity: %f\nCalculated: %f" % (porosity,porosity_from_volumes(volumes)))

In [ ]:
assert round(porosity,2) == round(porosity_from_volumes(volumes),2)

### Simple test with two x-values (empty and filled)

In [ ]:
cutoff_Si = 1000.0
max_cutoff_caps_test = [c_silicon_cutoff, max_cutoff_cap_graphite, max_cutoff_cap_dead]

x1,x2 =  createX(max_cutoff_caps_test, max_caps, limits = [0.0, 1.0], bits = 2, option=None)

print( x1)
print( x2)

V1 = volume_vs_x(x1, volumes, expansion_coeffs, eps)
C1 = capacity_all(x1,max_caps, volumes, densities)
C1_Si = capacity(x1[0],maxcap_si)

V2 = volume_vs_x(x2, volumes, expansion_coeffs, eps)
C2 = capacity_all(x2,max_caps, volumes, densities)
C2_Si = capacity(x2[0],maxcap_si)
#print volumes

print( "\nvolumes (Si, Graphite, Dead, Void):")
print( V1)
print( "x = %3.2f, P = %3.2f, C = %5.2f mAh/g, C_Si = %5.2f mAh/g(Si)" % (x1[0], porosity_from_volumes(V1), C1, C1_Si))

print( "\nvolumes (Si, Graphite, Dead, Void):")
print( V2)
print( "x = %3.2f, P = %3.2f, C = %5.2f mAh/g, C_Si = %5.2f mAh/g(Si)" % (x2[0], porosity_from_volumes(V2), C2, C2_Si))


assert round(C2_Si,2) == round(c_silicon_cutoff,2)

### Full range

In [ ]:
# variables 
bits = 100
cutoff_Si = 1000.0
porosity = 0.5
eps = 1.1

# creating cutoff list
max_cutoff_caps_test = [c_silicon_cutoff, max_cutoff_cap_graphite, max_cutoff_cap_dead]

# calculating the initial pore volume (void volume) and creating volume list
volume_void_0 = pore_volume_from_porosity(porosity, [volume_silicon_0,volume_graphite_0, volume_dead_0])
volumes = [volume_silicon_0,volume_graphite_0,volume_dead_0, volume_void_0]

# creating fraction range (x)
X =  createX(max_cutoff_caps_test, max_caps, limits = [0.0, 1.0], bits = bits, option="scalingSi")
_x    = np.linspace(0.0,1.0,bits)
_c_Si = capacity(X[:,0],maxcap_si)


# setting up initial numpy arrays
cols = 4
rows = bits

V = np.zeros((rows,cols))
P = np.zeros((rows,1))
C = np.zeros((rows,1))

# iterating through the x-values and calculating V,P, and C
row_no = 0
for x in X:
    V[row_no,:] = volume_vs_x(x, volumes, expansion_coeffs, eps)
    P[row_no] = porosity_from_volumes(V[row_no, :])
    C[row_no] = capacity_all(x,max_caps, volumes, densities)
    row_no += 1

# report
print( "Maximum capacity: %5.2f mAh/g(Tot)" % (C[-1]))
print( "Initial porosity: %5.2f" % (porosity))
print( "Final porosity:   %5.2f" % (P[-1]),)
if eps > 1.0:
    print( "(including volume for expansion (eps>1))")
else:
    print()

# replace negative values with np.NAN
V = np.where(V>0.0,V, np.NAN) # location (indexes) where (V > 0.0); if TRUE, return V[location], else return np.NAN
P = np.where(P>0.0,P, np.NAN)
C = np.where(C>0.0,C, np.NAN)

# plotting
plt.plot(_x,V[:,0], label = "Silicon")
plt.plot(_x,V[:,1], label = "Graphite")
plt.plot(_x,V[:,2], label = "Dead")
plt.plot(_x,V[:,3], label = "Void")
plt.xlabel("fraction")
plt.ylabel("Volume")
plt.legend()
print()
print( "Volume vs. Fraction".upper())



More plots...

In [ ]:
plt.plot(_x,P)
plt.xlabel("fraction")
plt.ylabel("Porosity")
print( "\nPorosity vs. Fraction".upper())

In [ ]:
plt.plot(_x,C)
plt.xlabel("fraction")
plt.ylabel("total capacity (mAh/g)")
print()
print( "Total capacity vs. Fraction".upper())

In [ ]:
plt.plot(C,P)
plt.xlabel("total capacity (mAh/g)")
plt.ylabel("porosity")
plt.axhline(0.4,linestyle = "--")
print()
print( "Porosity vs. total capacity".upper())

In [ ]:
plt.plot(_c_Si,P)
plt.xlabel("capacity (mAh/g(Si))")
plt.ylabel("porosity")
plt.axhline(0.4,linestyle = "--")
print()
print( "Porosity vs. capacity pr gram Silicon".upper())


In [ ]:
c0 = 320.0  # mAh/g(Si)
c1 = 3600.0 # mAh/g(Si)
l  = 2.5    # mAh/cm2



def find_c_for_p_cutoff(X, p_cutoff, volumes, expansion_coeffs, eps, max_caps, densities):
    for x in X:
        V = volume_vs_x(x, volumes, expansion_coeffs, eps)
        P = porosity_from_volumes(V)
        if P<=p_cutoff:
            return capacity_all(x,max_caps, volumes, densities)
    #return capacity_all(x,max_caps, volumes, densities)
    return np.nan


# variables 
bits = 100
cutoff_Si = 1000.0
porosity_cut_off = 0.4
eps = 1.0

# creating cutoff list
max_cutoff_caps_test = [c_silicon_cutoff, max_cutoff_cap_graphite, max_cutoff_cap_dead]

# creating fraction range (x) and starting porosities
X =  createX(max_cutoff_caps_test, max_caps, limits = [0.0, 1.0], bits = bits, option="scalingSi")
_p = np.linspace(0.45,0.7,20)
#_p = [0.6,0.5]

V  = [[volume_silicon_0,
       volume_graphite_0,
       volume_dead_0, 
       pore_volume_from_porosity(p, [volume_silicon_0,volume_graphite_0, volume_dead_0])] for p in _p]
c  = [find_c_for_p_cutoff(X, porosity_cut_off,v, expansion_coeffs, eps, max_caps, densities) for v in V]

plt.plot(_p,c,)
plt.xlabel("starting porosity")
plt.ylabel("capacity reach before \nporosity decreased to cut-off-value")
plt.axvline(porosity_cut_off,linestyle = '--', label = "cut-off")
plt.xlim([0.2, 0.8])
plt.ylim([200, 1000])
plt.legend()

In [ ]:
f_silicon  = 0.522
f_graphite = 0.13
f_dead     = 1.0 - (f_silicon + f_graphite)
print( f_dead)


In [ ]:
c0 = 320.0  # mAh/g(Si)
c1 = 3600.0 # mAh/g(Si)
l  = 2.5    # mAh/cm2



def find_c_for_p_cutoff(X, p_cutoff, volumes, expansion_coeffs, eps, max_caps, densities):
    for x in X:
        V = volume_vs_x(x, volumes, expansion_coeffs, eps)
        P = porosity_from_volumes(V)
        if P<=p_cutoff:
            return capacity_all(x,max_caps, volumes, densities)
    #return capacity_all(x,max_caps, volumes, densities)
    return np.nan


def find_v_for_porosity(porosity,v):
    V = []
    for p in porosity:
        v_void = pore_volume_from_porosity(p, v)
        V.append([v[0],v[1], v[2],v_void])
    return V

def get_volumes_from_fractions(MF,RHO):
    V = [volume_from_fR(mf,rho) for mf,rho in zip(MF,RHO)]
    return V
    

# variables 
fname =r"C:\Scripting\MyFiles\notebooks\ElectrodeCalculations\SiliconExpansion\CvsPvsF_dead_0_348.png"
bits = 100
porosity_cut_off = 0.4
eps = 1.0
f_dead     = 0.348
M   = [maxcap_si, max_cutoff_cap_graphite, max_cutoff_cap_dead]
RHO = [rho_si, rho_graphite, rho_dead]

_p = np.linspace(0.45,0.8,20)
#f_silicon = np.linspace(0.1,round(1.0-f_dead,2),5)
f_silicon = [0.1,0.2, 0.3, 0.4, 0.5, 0.6]
fig = plt.figure()
fig.suptitle("Capacity reached before porosity decreases to cut-off-value")
for f in f_silicon:
    MF  = [f, 1-f-f_dead,f_dead]
    V   = get_volumes_from_fractions(MF,RHO)
    X =  createX(M, max_caps, limits = [0.0, 1.0], bits = bits, option="scalingSi")
    c  = [find_c_for_p_cutoff(X, porosity_cut_off,v, expansion_coeffs, eps, max_caps, RHO) for v in find_v_for_porosity(_p,V)]

    plt.plot(_p,c, label = "%3.0f%% Si" % (100.0*f))
#plt.title("Capacity reach before porosity decreases to cut-off-value")
plt.xlabel("Starting porosity")
plt.ylabel("Capacity reach (mAh/g(tot))")
plt.xlim([_p[0]-0.05, _p[-1]+0.15])
plt.ylim([0.0, 2500.0])
plt.grid()
plt.legend()
plt.savefig(fname, dpi = 500)
print( "plotting the capacity reached when starting with a given porosity for different fractions of Si".upper())
print( 100*"-")
print( "contains Si, Graphite and dead material")
print( "mass fraction of dead material (CB, BINDER, OTHER): %3.2f" % f_dead)
print( "porosity cut-off: %3.2f" % (porosity_cut_off))

## Calculations for double-binder electrodes
 - Name: xx0nn (not disclosed)

|Material         |      fraction |
|-----------------|---------------|
|Si (xxx (Elkem)) | 0.5218        |
|graphite (SLP 30)| 0.0868        |
|CB (C65)         | 0.1308        |
|B1 (xxx)         | 0.0653        |
|B2 (xxx)         | 0.0656        |
|Buffer chemicals | 0.1298        |




In [ ]:
# defining mass fractions for xx0nn
f_binder_1 = 0.0653
f_binder_2 = 0.0656
f_silicon_xx0nn = 0.5218
f_graphite_xx0nn = 0.0868
f_cb_xx0nn = 0.1308
f_chemicals = 0.1298
f_binder_xx0nn = f_binder_1 + f_binder_2
f_dead_xx0nn = f_binder_xx0nn + f_cb_xx0nn + f_chemicals

mass_fractions_xx0nn = [f_silicon_xx0nn,f_graphite_xx0nn,f_dead_xx0nn]

assert round(np.sum(mass_fractions_xx0nn),3) == 1.0


# other variablse
fname1 =r"C:\Scripting\MyFiles\notebooks\ElectrodeCalculations\SiliconExpansion\CvsP_xx0nn.png"
fname2 =r"C:\Scripting\MyFiles\notebooks\ElectrodeCalculations\SiliconExpansion\CvsP_SiCap_xx0nn.png"
bits = 100
porosity_cut_off = 0.4
eps = 1.0
cutoff_Si = 1000.0


# creating cutoff list
max_cutoff_caps_xx0nn = [cutoff_Si, max_cutoff_cap_graphite, max_cutoff_cap_dead]


# need to make the initial volume list
volume_silicon_0 = volume_from_fR(f_silicon_xx0nn,rho_si)
volume_graphite_0 = volume_from_fR(f_graphite_xx0nn,rho_graphite)
volume_dead_0 = volume_from_fR(f_dead_xx0nn,rho_dead)


# creating fraction range (x) and starting porosities
X =  createX(max_cutoff_caps_xx0nn, max_caps, limits = [0.0, 1.0], bits = bits, option="scalingSi")
cSi =  capacity(X[:,0],maxcap_si)


_p = np.linspace(0.32,0.85,100)

V  = [[volume_silicon_0,
       volume_graphite_0,
       volume_dead_0, 
       pore_volume_from_porosity(p, [volume_silicon_0,volume_graphite_0, volume_dead_0])] for p in _p]
c   = [find_c_for_p_cutoff(X, porosity_cut_off,v, expansion_coeffs, eps, max_caps, densities) for v in V]
cSi = np.array(c)/f_silicon_xx0nn

fig = plt.figure()
#fig.suptitle("Capacity reached before porosity decreases to cut-off-value")
plt.plot(_p,c,)
plt.xlabel("starting porosity")
plt.ylabel("capacity (mAh/g(tot))")
#plt.xlim([0.1, 0.8])
#plt.ylim([200, 1000])
plt.grid()
plt.legend()
plt.savefig(fname1, dpi = 500)

fig = plt.figure()
#fig.suptitle("Capacity reached before porosity decreases to cut-off-value")
plt.plot(_p,cSi)
plt.xlabel("starting porosity")
plt.ylabel("capacity (mAh/g(Si))")
plt.grid()
plt.axhline(1000.0, linestyle = "--")
index = np.where(cSi[~np.isnan(cSi)]>=(1000.0+0.0001))
i = index[-1][0]
p_i = _p[i]
p_i_txt = "p = %3.2f" % p_i

plt.axvline(p_i, linestyle = "--")
#plt.text(0.55, 1100.0, p_i_txt)

plt.annotate(p_i_txt, xy=(p_i, 1000), xytext=(0.46, 1100),
             arrowprops=dict(facecolor='black', shrink=0.05, width = 1.5, frac = 0.3))
plt.savefig(fname2, dpi = 500)
print( "saved to:")
print( fname1)
print( fname2)

print()
c1 = 1000.0 # mAh/g(Si)
l  = 2.5    # mAh/cm2
p1 = 0.56



d = density_all(densities, mass_fractions_xx0nn, porosity=p_i)
print( calc_needed_thickness(f_silicon_xx0nn*cutoff_Si,l,d)*to_micron)

In [ ]:
!jupyter nbconvert --to html 2016_04_VolumeConstrainsGraphiteSiElectrode_py36.ipynb